In [1]:
import numpy as np
import re
from operator import itemgetter

In [2]:
Corpus = {
    'l o w _':5,#frequency
    'l o w e r _':2,
    'n e w e s t _':6,
    'w i d e s t _':3,
    'h a p p i e r _':2
}

In [4]:
Corpus

{'l o w _': 5,
 'l o w e r _': 2,
 'n e w e s t _': 6,
 'w i d e s t _': 3,
 'h a p p i e r _': 2}

In [61]:
def getPairCounts(Corpus):
    pairs = {}
    for word,fr in Corpus.items():
        symbols = word.split(' ')
        for i in range(len(symbols)-1):
            pair = (symbols[i],symbols[i+1])
            cfr = pairs.get(pair,0)
            pairs[pair] = cfr+fr
    return pairs

In [72]:
getPairCounts(Corpus)

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', '_'): 5,
 ('w', 'e'): 8,
 ('e', 'r'): 4,
 ('r', '_'): 4,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('t', '_'): 9,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'i'): 2,
 ('i', 'e'): 2}

In [73]:
pairsCounts = getPairCounts(Corpus)

In [64]:
# print(pairsCounts)
pairsCounts


{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', '_'): 5,
 ('w', 'e'): 8,
 ('e', 'r'): 4,
 ('r', '_'): 4,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('t', '_'): 9,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'i'): 2,
 ('i', 'e'): 2}

In [74]:
def getBestPair(pairsCounts):
    return max(pairsCounts,key=pairsCounts.get)

In [76]:
print(getBestPair(pairsCounts))

('e', 's')


In [77]:
def mergeInCorpus(bestPair,Corpus):
    newCorpus = {}
    for word in Corpus:
        newWord = re.sub(' '.join(bestPair),''.join(bestPair),word)#pattern,replacement,word
        newCorpus[newWord] = Corpus[word]
    return newCorpus

In [79]:
bestPair = getBestPair(pairsCounts)
print(bestPair)

('e', 's')


In [80]:
newCorpus = mergeInCorpus(bestPair,Corpus)
# print(newCorpus)
newCorpus

{'l o w _': 5,
 'l o w e r _': 2,
 'n e w es t _': 6,
 'w i d es t _': 3,
 'h a p p i e r _': 2}

## Binary Pair Encoding

In [81]:
def runBPE(Corpus,k):
    bpeStats = {}
    for i in range(k):
        pairsCounts = getPairCounts(Corpus)
        if not pairsCounts:
            break
        bestPair = getBestPair(pairsCounts)
        bpeStats[bestPair] = i
        Corpus = mergeInCorpus(bestPair,Corpus)
    return Corpus,bpeStats

In [83]:
runBPE(Corpus,10)[0]

{'low_': 5, 'low er _': 2, 'newest_': 6, 'w i d est_': 3, 'h a p p i er _': 2}

In [84]:
runBPE(Corpus,10)[1]

{('e', 's'): 0,
 ('es', 't'): 1,
 ('est', '_'): 2,
 ('l', 'o'): 3,
 ('lo', 'w'): 4,
 ('n', 'e'): 5,
 ('ne', 'w'): 6,
 ('new', 'est_'): 7,
 ('low', '_'): 8,
 ('e', 'r'): 9}

In [85]:
def getAllPairs(word):
    pairs = []
    word = word.split(' ')
    prevChar = word[0]
    for char in word[1:]:
        pairs.append((prevChar,char))
        prevChar = char
    return pairs

In [89]:
newWord = 'lowest'
newWord2 = ' '.join(list(newWord))+' _'
print(newWord,newWord2)

lowest l o w e s t _


In [90]:
pairs = getAllPairs(newWord2)
pairs

[('l', 'o'), ('o', 'w'), ('w', 'e'), ('e', 's'), ('s', 't'), ('t', '_')]

In [94]:
newCorpus,bpeStats = runBPE(Corpus,10)

In [95]:
def getPairToBeMerged(bpeStats,pairs):
    #bpeCodes = [(pair,bpeStats[pair]) for pair in pairs if pair in bpeStats]
    bpeCodes = []
    for pair in pairs:
        if pair in bpeStats:
            bpeCodes.append((pair,bpeStats[pair]))
    if len(bpeCodes) == 0:
        return (-1,-1)
    pairToBeMerged = min(bpeCodes,key=itemgetter(1))[0]
    return pairToBeMerged

In [96]:
pairToBeMerged = getPairToBeMerged(bpeStats,pairs)

In [97]:
pairToBeMerged

('e', 's')

In [98]:
def mergeLetters(word,pairToBeMerged):
    newWord = re.sub(' '.join(pairToBeMerged),''.join(pairToBeMerged),word)
    return newWord

In [99]:
print(mergeLetters(newWord2,pairToBeMerged))

l o w es t _


In [105]:
def bpeTokenize(word,bpeStats):
    if len(word) == 1:
        return word
    word = ' '.join(list(word))+' _'
    while True:
        pairs = getAllPairs(word)
        pairToBeMerged = getPairToBeMerged(bpeStats,pairs)
        if pairToBeMerged[0] == -1:
            break
        word = mergeLetters(word,pairToBeMerged)
    return word

In [106]:
newWord = bpeTokenize('lowest',bpeStats)

In [107]:
newWord

'low est_'